# Table of Contents
<p>
<div class="lev1 toc-item">
    <a href="#Cargamos-transacciones-de-2016" data-toc-modified-id="Cargamos-transacciones-de-2016">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>
        Cargamos transacciones de 2016</a></div>

<div class="lev1 toc-item">
    <a href="#Limpieza-y-transformación-de-transacciones" data-toc-modified-id="Limpieza-y-transformación-de-transacciones">
    <span class="toc-item-num">2&nbsp;&nbsp;</span>
    Limpieza y transformación de transacciones</a></div>
    
<div class="lev1 toc-item">
    <a href="#Cargamos-datos-de-ubicación-geográfica" data-toc-modified-id="Cargamos-datos-de-ubicación-geográfica">
    <span class="toc-item-num">3&nbsp;&nbsp;</span>
    Cargamos datos de ubicación geográfica</a></div>
    
<div class="lev1 toc-item">
    <a href="#Cargamos-datos-de-capacidad-de-plazas" data-toc-modified-id="Cargamos-datos-de-capacidad-de-plazas">
    <span class="toc-item-num">4&nbsp;&nbsp;</span>
    Cargamos datos de capacidad de plazas</a></div>

<div class="lev1 toc-item">
    <a href="#Relacionamos-transacciones,-ubicaciones-y-capacidad" data-toc-modified-id="Relacionamos-transacciones,-ubicaciones-y-capacidad">
    <span class="toc-item-num">5&nbsp;&nbsp;</span>
    Relacionamos transacciones, ubicaciones y capacidad</a></div>
          
<div class="lev1 toc-item">
    <a href="#Construimos-la-serie-de-ocupación-por-timestamp" data-toc-modified-id="Construimos-la-serie-de-ocupación-por-timestamp">
    <span class="toc-item-num">6&nbsp;&nbsp;</span>
    Construimos la serie de ocupación por timestamp</a></div>
    
<div class="lev1 toc-item">
    <a href="#Completamos-la-serie-con-datos-meteorológicos" data-toc-modified-id="Completamos-la-serie-con-datos-meteorológicos">
    <span class="toc-item-num">7&nbsp;&nbsp;</span>
    Completamos la serie con datos meteorológicos</a></div>

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
% matplotlib inline

## Cargamos transacciones de 2016

In [2]:
df = pd.read_csv(os.path.join('./data/ParkingTransaction_2016_cleaned.csv'), 
                           parse_dates=['TransactionDateTime'])

In [3]:
df['TransactionDateTime'].describe()

count                10935395
unique                5505469
top       2016-04-08 12:03:59
freq                       63
first     2016-01-01 11:49:15
last      2016-12-31 20:00:51
Name: TransactionDateTime, dtype: object

In [4]:
import re

_underscorer1 = re.compile(r'(.)([A-Z][a-z]+)')
_underscorer2 = re.compile('([a-z0-9])([A-Z])')

def camelToSnake(s):
    subbed = _underscorer1.sub(r'\1_\2', s)
    return _underscorer2.sub(r'\1_\2', subbed).lower()

In [5]:
df.columns = df.columns.map(lambda x: camelToSnake(x))

df.transaction_date = pd.to_datetime(df.transaction_date, format="%Y-%m-%d")
df.sort_values('transaction_date_time', inplace=True)

# creamos nueva columna 'final_date_time'
df['final_date_time'] = df.transaction_date_time.add(pd.to_timedelta(df.duration_mins, unit="m"))

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10935395 entries, 8 to 10912795
Data columns (total 12 columns):
unnamed: 0               int64
transaction_id           int64
transaction_date_time    datetime64[ns]
transaction_date         datetime64[ns]
time_start               object
time_expired             object
duration_mins            int64
amount                   float64
payment_mean             object
meter_code               int64
element_key              int64
final_date_time          datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(5), object(3)
memory usage: 2.8 GB


In [6]:
df = df.drop(columns=['unnamed: 0','transaction_date','amount','payment_mean',
                      'meter_code','time_start','time_expired'])

In [7]:
df.head()

,transaction_id,transaction_date_time,duration_mins,element_key,final_date_time
8,183506786,2016-01-01 11:49:15,0,54730,2016-01-01 11:49:15
4,183511748,2016-01-01 12:36:04,0,88773,2016-01-01 12:36:04
17,183511753,2016-01-01 12:36:08,0,88773,2016-01-01 12:36:08
15,183529739,2016-01-01 12:40:30,0,57354,2016-01-01 12:40:30
12,183527118,2016-01-01 14:52:45,0,76102,2016-01-01 14:52:45


In [8]:
df.tail()

,transaction_id,transaction_date_time,duration_mins,element_key,final_date_time
10909957,313159887,2016-12-31 19:58:49,2,81194,2016-12-31 20:00:49
10918741,313159702,2016-12-31 19:58:53,2,79570,2016-12-31 20:00:53
10910192,313159980,2016-12-31 19:59:24,1,1234,2016-12-31 20:00:24
10907062,313159989,2016-12-31 19:59:45,0,43914,2016-12-31 19:59:45
10912795,313160195,2016-12-31 20:00:51,0,8306,2016-12-31 20:00:51


### Limpieza y transformación de transacciones

In [9]:
df.transaction_id.duplicated().sum()  # no hay transacciones duplicadas

0

In [10]:
df.isnull().sum()  # no hay valores nulos

transaction_id           0
transaction_date_time    0
duration_mins            0
element_key              0
final_date_time          0
dtype: int64

#### Duración incorrecta

In [11]:
# Observamos algunas transacciones con duración = 0
df.duration_mins.loc[df.duration_mins == 0].count()

9086

In [12]:
# Incluso hay algunas transacciones con duración negativa
df.duration_mins.loc[df.duration_mins < 0].count()

1118

In [13]:
# Eliminamos ambos casos que suponen menos de un 0.1% del total
df = df.loc[df.duration_mins > 0]

#### Transacciones de larga duración

In [14]:
# Procesado de transacciones con distinta fecha de inicio y fin
long_trans = df.loc[df.transaction_date_time.dt.date != df.final_date_time.dt.date]
indexes = df.loc[df.transaction_date_time.dt.date != df.final_date_time.dt.date].index.get_values()
df = df.drop(index=indexes, axis=1)

In [15]:
long_trans_dup = long_trans.copy()
long_trans_dup = long_trans_dup.loc[long_trans_dup.final_date_time.dt.hour >= 8]
long_trans.final_date_time = long_trans.transaction_date_time.apply(
    lambda x: x.replace(hour=20,minute=0,second=0,microsecond=0))
long_trans_dup.transaction_date_time = long_trans_dup.final_date_time.apply(
    lambda x: x.replace(hour=8,minute=0,second=0,microsecond=0))

In [16]:
long_trans_full = pd.concat([long_trans,long_trans_dup], ignore_index=True, sort=True)
df = pd.concat([df,long_trans_full], ignore_index=True, sort=True)

#### Horario de funcionamiento de los parquímetros

In [17]:
# Eliminamos las transacciones con inicio y fin antes de las 08:00
indexes = df.loc[(df.transaction_date_time.dt.hour < 8) & (df.final_date_time.dt.hour < 8)].index.get_values()
df = df.drop(index=indexes, axis=1)

In [18]:
# Eliminamos las transacciones con inicio y fin después de las 20:00
indexes = df.loc[(df.transaction_date_time.dt.hour >= 20) & (df.final_date_time.dt.hour >= 20)].index.get_values()
df = df.drop(index=indexes, axis=1)

In [19]:
# Redondeamos la hora (sin minutos o segundos) y fijo a 08:00 la hora mínima de comienzo
df.transaction_date_time = np.where(df.transaction_date_time.dt.hour < 8, 
                                    df.transaction_date_time.apply(lambda x: x.replace(hour=8,minute=0,second=0,microsecond=0)), 
                                    df.transaction_date_time.apply(lambda x: x.replace(minute=0,second=0,microsecond=0)))

In [20]:
# Redondeamos la hora final (sin minutos o segundos) y fijo a 20:00 la hora máxima final
df.final_date_time = np.where(df.final_date_time.dt.hour > 20,
                              df.final_date_time.apply(lambda x: x.replace(hour=20,minute=0,second=0,microsecond=0)), 
                              df.final_date_time.apply(lambda x: x.replace(minute=0,second=0,microsecond=0)))

In [21]:
# Comprobamos que no hay transacciones fuera del rango horario de uso de los parquímetros (de 8 a 20h)
df.element_key.loc[((df.transaction_date_time.dt.hour > 20) | (df.transaction_date_time.dt.hour < 8)) | 
       ((df.final_date_time.dt.hour > 20) | (df.final_date_time.dt.hour < 8))].count()

0

#### Domingos o festivos

In [22]:
# Observamos algunas transacciones realizadas por error en domingo
df.transaction_date_time.loc[df.transaction_date_time.dt.weekday == 6].count()

239

In [23]:
# Las eliminamos también
df = df.loc[df.transaction_date_time.dt.weekday != 6]

In [24]:
# Y observamos transacciones en días festivos
import holidays

hol = holidays.US(state='WA', years=[2016]).items()
hol_dates = []
for dat, name in sorted(hol):
    hol_dates.append(dat)

df.transaction_date_time.loc[df.transaction_date_time.dt.date.isin(hol_dates)].count()

32871

In [25]:
# Las eliminamos también
df = df.loc[~df.transaction_date_time.dt.date.isin(hol_dates)]

In [26]:
df.shape  

(10822236, 5)

In [27]:
# Con la limpieza hemos reducido el tamaño del dataset inicial en poco más de un 1%
(10935395-(df.shape[0]-long_trans_dup.shape[0]))/10935395*100

1.0592575759723357

## Cargamos datos de ubicación geográfica

In [28]:
coord = pd.read_csv(os.path.join('./data/Coord_EK.csv'))

In [29]:
coord.head()

,element_key,latitude,longitude
0,1001,47.602862,-122.334703
1,1002,47.602997,-122.334538
2,1005,47.603602,-122.335382
3,1006,47.603725,-122.335171
4,1009,47.605010,-122.336669


In [30]:
coord.duplicated().sum()  # no hay registros duplicados

0

In [31]:
coord.element_key.duplicated().sum()  # tampoco hay duplicados de Element Key

0

In [32]:
coord[['latitude','longitude']].duplicated().sum()  # ni hay ubicaciones repetidas

0

## Cargamos datos de capacidad de plazas

In [33]:
park_spaces = pd.read_csv(os.path.join('./data/StreetParking.csv'))

In [34]:
park_spaces.head()

,element_key,parking_category,total_nopark,total_zones,parking_spaces,total_spaces
0,32018,Paid Parking,8,1,6,15
1,1042,No Parking Allowed,5,1,0,6
2,48129,Paid Parking,6,1,4,11
3,47962,Paid Parking,8,2,7,17
4,47966,Paid Parking,5,1,4,10


In [35]:
park_spaces.total_spaces.describe().astype(str)

count               46101.0
mean     14.541832064380381
std      10.353842357254363
min                     0.0
25%                     9.0
50%                    12.0
75%                    19.0
max                   202.0
Name: total_spaces, dtype: object

In [36]:
park_spaces.parking_spaces.describe().astype(str)

count               46101.0
mean     11.147588989392855
std      10.140466606852842
min                     0.0
25%                     5.0
50%                     9.0
75%                    15.0
max                   202.0
Name: parking_spaces, dtype: object

In [37]:
park_spaces.parking_spaces.sort_values(ascending=False).head(10)  # el valor máximo no es un outlier

27277    202
19307    198
5369     171
22653    171
26435    166
7952     166
29247    154
9009     154
1637     141
19337    141
Name: parking_spaces, dtype: int64

In [38]:
park_spaces.element_key.loc[park_spaces.parking_spaces == 0].count()

4914

In [39]:
# Eliminamos los registros que no tienen información de la capacidad de plazas
park_spaces = park_spaces.loc[park_spaces.parking_spaces != 0]

In [40]:
park_spaces.shape

(41187, 6)

In [41]:
park_spaces.head()

,element_key,parking_category,total_nopark,total_zones,parking_spaces,total_spaces
0,32018,Paid Parking,8,1,6,15
2,48129,Paid Parking,6,1,4,11
3,47962,Paid Parking,8,2,7,17
4,47966,Paid Parking,5,1,4,10
5,8617,Paid Parking,7,2,4,13


## Relacionamos transacciones, ubicaciones y capacidad

In [42]:
df.element_key.unique().size

1514

In [43]:
coord.element_key.unique().size

1517

In [44]:
park_spaces.element_key.unique().size

41187

In [45]:
len(set(coord.element_key).intersection(set(df.element_key)))

1445

In [46]:
len(set(park_spaces.element_key).intersection(set(df.element_key)))

1428

**Mezclamos los datasets:**

In [47]:
df = df.drop(columns=['duration_mins'])

In [48]:
# Transacciones con Coordenadas
df_c = pd.merge(df, coord, on='element_key', how='inner', validate='many_to_one')

In [49]:
# Transacciones con Coordenadas y con Capacidad de Plazas disponibles
df_cp = pd.merge(df_c, park_spaces, on='element_key', how='inner', validate='many_to_one')

In [50]:
df.shape, df_c.shape, df_cp.shape

((10822236, 4), (10657805, 6), (10574270, 11))

In [51]:
#df_cp.set_index('transaction_id', inplace=True)
df_cp.head()

,element_key,final_date_time,transaction_date_time,transaction_id,latitude,longitude,parking_category,total_nopark,total_zones,parking_spaces,total_spaces
0,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64059668,47.619158,-122.346457,Paid Parking,5,1,7,13
1,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64059669,47.619158,-122.346457,Paid Parking,5,1,7,13
2,35693,2016-01-02 14:00:00,2016-01-02 08:00:00,64084672,47.619158,-122.346457,Paid Parking,5,1,7,13
3,35693,2016-01-02 09:00:00,2016-01-02 08:00:00,183563751,47.619158,-122.346457,Paid Parking,5,1,7,13
4,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64084673,47.619158,-122.346457,Paid Parking,5,1,7,13


In [52]:
df_cp[['element_key','parking_category']].groupby(by='parking_category').count()/df_cp.shape[0]

,element_key
parking_category,
Carpool Parking,0.010041
Paid Parking,0.967317
Restricted Parking Zone,0.022642


In [53]:
# Nos quedamos con los parquímetros de los bloques de la categoría mayoritaria
df_cp = df_cp.loc[df_cp.parking_category == 'Paid Parking']

In [54]:
df_cp.element_key.unique().size

1290

In [55]:
# Eliminamos aquellos parquímetros con un número pequeño de plazas
df_cp.element_key.loc[df_cp.total_spaces < 8].unique().size
df_cp = df_cp.loc[df_cp.total_spaces > 8]

# Construimos la serie de ocupación por timestamp

Construimos primero la tabla corta que usaremos para montar las series temporales:

In [56]:
in_cols = ['element_key', 'latitude', 'longitude', 'transaction_date_time', 'parking_spaces','total_spaces']
out_cols = ['element_key', 'latitude', 'longitude', 'final_date_time', 'parking_spaces','total_spaces']

df_in = df_cp[in_cols].reset_index(drop=True)
#df_in.transaction_id = df_in.transaction_id.map(lambda s:'%s_in' % str(s))
df_in['timestamp_sign'] = 1.0
df_in.rename(columns={'transaction_date_time': 'timestamp'}, inplace=True)

df_out = df_cp[out_cols].reset_index(drop=True)
#df_out.transaction_id = df_out.transaction_id.map(lambda s:'%s_out' % str(s))
df_out['timestamp_sign'] = - 1.0
df_out.rename(columns={'final_date_time': 'timestamp'}, inplace=True)  # inplace = True, value of copy is ignored

In [57]:
df_in.head()

,element_key,latitude,longitude,timestamp,parking_spaces,total_spaces,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 08:00:00,7,13,1.0
1,35693,47.619158,-122.346457,2016-01-02 08:00:00,7,13,1.0
2,35693,47.619158,-122.346457,2016-01-02 08:00:00,7,13,1.0
3,35693,47.619158,-122.346457,2016-01-02 08:00:00,7,13,1.0
4,35693,47.619158,-122.346457,2016-01-02 08:00:00,7,13,1.0


In [58]:
df_out.head()

,element_key,latitude,longitude,timestamp,parking_spaces,total_spaces,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 10:00:00,7,13,-1.0
1,35693,47.619158,-122.346457,2016-01-02 10:00:00,7,13,-1.0
2,35693,47.619158,-122.346457,2016-01-02 14:00:00,7,13,-1.0
3,35693,47.619158,-122.346457,2016-01-02 09:00:00,7,13,-1.0
4,35693,47.619158,-122.346457,2016-01-02 10:00:00,7,13,-1.0


In [59]:
full_transactions = pd.concat([df_in, df_out])
full_transactions.sort_values('timestamp', ascending=True, inplace=True)

In [60]:
full_transactions.head()

,element_key,latitude,longitude,timestamp,parking_spaces,total_spaces,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 08:00:00,7,13,1.0
265338,33741,47.619820,-122.347160,2016-01-02 08:00:00,4,12,1.0
673956,9353,47.612938,-122.344920,2016-01-02 08:00:00,16,20,1.0
673957,9353,47.612938,-122.344920,2016-01-02 08:00:00,16,20,1.0
673958,9353,47.612938,-122.344920,2016-01-02 08:00:00,16,20,1.0


In [61]:
df_in.shape, df_out.shape, full_transactions.shape

((9851701, 7), (9851701, 7), (19703402, 7))

#### Añadimos porcentaje de ocupación

In [62]:
# COMANDO A REVISAR OPTIMIZACIÓN - LE HA COSTADO
full_transactions['occupation'] = full_transactions.groupby(['element_key', 'timestamp']).timestamp_sign.transform(
    lambda series: series.cumsum())

In [63]:
full_transactions = full_transactions.reset_index(drop=True)

In [64]:
full_transactions.drop_duplicates(subset=['element_key','timestamp'], keep='last', inplace=True)

In [65]:
full_transactions['day_year'] = full_transactions.timestamp.dt.dayofyear

In [66]:
full_transactions['occu_cum'] = full_transactions.groupby(['element_key','day_year']).occupation.agg('cumsum')

In [67]:
full_transactions['occu_perc_ps'] = (full_transactions.occu_cum / full_transactions.parking_spaces * 100.0).map(
    "{0:.2f}".format).astype(float)

In [68]:
full_transactions['occu_perc_ts'] = (full_transactions.occu_cum / full_transactions.total_spaces * 100.0).map(
    "{0:.2f}".format).astype(float)

In [69]:
full_transactions.element_key.loc[full_transactions.occu_perc_ps > 100].count()

263974

In [70]:
full_transactions.element_key.loc[full_transactions.occu_perc_ps > 100].unique().size

1116

In [71]:
full_transactions.element_key.loc[full_transactions.occu_perc_ts > 100].count()

5261

In [72]:
full_transactions.element_key.loc[full_transactions.occu_perc_ts > 100].unique().size

138

In [73]:
# Eliminamos aquellos parquímetros con porcentaje de ocupación superior al 100%
full_transactions = full_transactions.loc[full_transactions.occu_perc_ts < 100]

## Completamos la serie con datos meteorológicos

In [74]:
meteo = pd.read_csv(os.path.join('./data/seattleWeather_1948-2017.csv'))
meteo.columns = meteo.columns.map(lambda x: camelToSnake(x))
meteo.date = pd.to_datetime(meteo.date, format="%Y-%m-%d")

meteo = meteo.loc[meteo['date'].dt.year == 2016]
meteo['day_year'] = meteo.date.dt.dayofyear

In [75]:
meteo.head()

,date,prcp,tmax,tmin,rain,day_year
24837,2016-01-01,0.00,46,28,False,1
24838,2016-01-02,0.00,42,25,False,2
24839,2016-01-03,0.02,40,31,True,3
24840,2016-01-04,0.15,38,35,True,4
24841,2016-01-05,0.11,46,36,True,5


In [76]:
full_transactions_meteo = pd.merge(full_transactions, 
                                   meteo, on='day_year', how='inner', validate='many_to_one')

In [78]:
full_transactions_meteo.rename(columns={'occu_perc_ts': 'occupation_perc'}, inplace=True)
full_transactions_meteo[['element_key','latitude','longitude','timestamp','occupation_perc',
                         'prcp','tmax','tmin']].to_csv('./data/Serie_Total2016.csv', index=False)